In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Initialization: importing the packages that we will use
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu' # Google colab offers time limited use of GPU for free

# Training parameters 
BATCH_SIZE = 64

In [3]:
# For loading the data
from PIL import Image
import os
import os.path

import torchvision
import torch.utils.data
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import random

In [4]:
# For constructing the network
import torch.optim as optim
import torch.utils.data
import torch.nn as nn
import torchvision.models as models
import torch.utils.data
import torch.backends.cudnn as cudnn

In [5]:
from tqdm import tqdm

In [6]:
# Solve the imshow dead kernel problem
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [7]:
'''
Start loading the data
'''
print('================== START LOADING DATA ==================')

================== START LOADING DATA ==================


In [8]:
path_drive = '/content/drive/My Drive/'

In [9]:
!cp /content/drive/MyDrive/Task4/pretrain_features.csv /content
!cp /content/drive/MyDrive/Task4/pretrain_labels.csv /content
!cp /content/drive/MyDrive/Task4/test_features.csv /content
!cp /content/drive/MyDrive/Task4/train_features.csv /content
!cp /content/drive/MyDrive/Task4/train_labels.csv /content

In [10]:
r1 = pd.read_csv('pretrain_features.csv')  # features
r2 = pd.read_csv('pretrain_labels.csv')  # labels


print(r1.shape)
print(r2.shape)

# Merge two dataframes
all_data_st = pd.merge(r2, r1, on='Id')
print(all_data_st.shape)
print(all_data_st.head(10))

# # Output to csv
# all_data_st.to_csv("training_set.csv", index=None)

(50000, 1002)
(50000, 2)
(50000, 1003)
   Id  lumo_energy                                             smiles  \
0   0    -3.111521  c1occ2c1c1ccc3cscc3c1c1ncc3cc(ccc3c21)-c1cccc2...   
1   1    -3.219118  C1C=c2c(cc3ncc4c5[SiH2]C=Cc5oc4c3c2=C1)-c1scc2...   
2   2    -3.114145  C1C=c2c3cccnc3c3c4c[nH]cc4c4cc(cnc4c3c2=C1)-c1...   
3   3    -3.161867  [SiH2]1C=Cc2c1csc2-c1cnc2c(c1)c1ccccc1c1cc3ccc...   
4   4    -3.687744        c1occ2c1c(cc1[se]c3ccncc3c21)-c1cccc2nsnc12   
5   5    -2.791261  [SiH2]1C=Cc2[nH]c3c(oc4cc(sc34)-c3scc4cc[se]c3...   
6   6    -3.688235          c1ccc(nc1)-c1cc2ncc3c4cnccc4sc3c2c2nsnc12   
7   7    -3.243368  C1C=c2c3cccnc3c3c4cocc4c4C=C(Cc4c3c2=C1)c1scc2...   
8   8    -3.508069  c1cc2csc(-c3cc4c5cscc5c5c6occc6c6cscc6c5c4c4ns...   
9   9    -3.440629  [SiH2]1C=c2c3cc(oc3c3c4cocc4c4ccc5cscc5c4c3c2=...   

   feature_0000  feature_0001  feature_0002  feature_0003  feature_0004  \
0           0.0           0.0           0.0           0.0           0.0   
1      

In [11]:
# all_data_st = all_data_st.sample(frac=1)
# print(all_data_st.head(10))
all_data_st_noidsmiles = all_data_st.drop(['Id', 'smiles'], axis=1)
print(all_data_st_noidsmiles.head(10))
all_data_array = np.array(all_data_st_noidsmiles)
all_data_tensor = torch.tensor(all_data_array)
print(all_data_tensor)

   lumo_energy  feature_0000  feature_0001  feature_0002  feature_0003  \
0    -3.111521           0.0           0.0           0.0           0.0   
1    -3.219118           0.0           0.0           0.0           0.0   
2    -3.114145           0.0           0.0           0.0           0.0   
3    -3.161867           0.0           0.0           0.0           0.0   
4    -3.687744           0.0           0.0           0.0           0.0   
5    -2.791261           0.0           0.0           0.0           1.0   
6    -3.688235           0.0           0.0           0.0           1.0   
7    -3.243368           0.0           0.0           0.0           0.0   
8    -3.508069           0.0           0.0           0.0           0.0   
9    -3.440629           0.0           0.0           0.0           0.0   

   feature_0004  feature_0005  feature_0006  feature_0007  feature_0008  ...  \
0           0.0           0.0           0.0           0.0           0.0  ...   
1           1.0          

In [12]:
torch.tensor(all_data_st_noidsmiles.iloc[0][1:], dtype=torch.double)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 

In [13]:
# For reproducbility
cudnn.benchmark = False
cudnn.deterministic = True
np.random.seed(1998)
torch.manual_seed(1998)
torch.cuda.manual_seed(1998)
torch.backends.cudnn.deterministic = True
random.seed(1998)

In [14]:
# # Get the feature of the training set
# def get_features(filename):
#     features = pd.read_csv('./'+filename)
#     print(features.head(12))
#     # train = train.sample(frac=1).reset_index(drop=True)
#     # print(train.head())
#     return features

# # Get the labels of the training set
# def get_labels(filename):
#     labels = pd.read_csv('./'+filename)
#     print(labels.head(12))
#     # train = train.sample(frac=1).reset_index(drop=True)
#     # print(train.head())
#     return labels

# # Get the features of the testing set
# def get_test_feature(filename):
#     test_features = pd.read_csv('./'+filename)
#     print(test_features.head(12))
#     # train = train.sample(frac=1).reset_index(drop=True)
#     # print(train.head())
#     return test_features

# pretrain_features = get_features('pretrain_features.csv')
# pretrain_labels = get_labels('pretrain_labels.csv')
# train_features = get_features('train_features.csv')
# train_labels = get_labels('train_labels.csv')
# test_features = get_test_feature('test_features.csv')
# print(pretrain_features.shape)
# print(pretrain_labels.shape)
# print(train_features.shape)
# print(train_labels.shape)
# print(test_features.shape)

In [15]:
'''
Start constructing the network
'''
print('================== START CONSTRUCTING NETWORK ==================')

================== START CONSTRUCTING NETWORK ==================


In [16]:
class FeatureExtract(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer_1 = nn.Linear(
            in_features=kwargs["input_shape"], out_features=512
        )
        self.encoder_hidden_layer_2 = nn.Linear(
            in_features=512, out_features=256
        )
        self.encoder_hidden_layer_3 = nn.Linear(
            in_features=256, out_features=128
        )
        self.encoder_hidden_layer_4 = nn.Linear(
            in_features=128, out_features=64
        )
        self.prediction_layer = nn.Linear(
            in_features=64, out_features=1
        )
 
    def forward(self, features):
        activation_1 = self.encoder_hidden_layer_1(features)
        activation_1 = torch.relu(activation_1)
        activation_2 = self.encoder_hidden_layer_2(activation_1)
        activation_2 = torch.relu(activation_2)
        activation_3 = self.encoder_hidden_layer_3(activation_2)
        activation_3 = torch.relu(activation_3)
        activation_4 = self.encoder_hidden_layer_4(activation_3)
        activation_4 = torch.relu(activation_4)
        output = self.prediction_layer(activation_4)
        return output

In [17]:
model = FeatureExtract(input_shape=1000).to(device)

In [18]:
# Construct the loss and optimizer
criterion = nn.MSELoss()

optimizer = optim.SGD(model.parameters(),
                            lr=0.0005,
                            momentum=0.9,
                            weight_decay=2e-3,#The value used in the paper is 1e-3
                            nesterov=True)

In [20]:
len(all_data_st_noidsmiles)

50000

In [21]:
training = all_data_st_noidsmiles.sample(frac=0.95, random_state=1998)

validation = all_data_st_noidsmiles.drop(training.index)

print(len(training))
print(len(validation))

47500
2500


In [24]:
input = torch.tensor(training.iloc[0][1:], dtype=torch.float).to(device)
result = model(input)
loss = criterion(result, torch.tensor(training.iloc[0][0:1], dtype=torch.float).to(device))
print(loss)

tensor(15.2968, device='cuda:0', grad_fn=<MseLossBackward0>)


In [25]:
def val(model, criterion, valset, datapoint):

  torch.cuda.empty_cache()
  
  validation_loss_sum = 0

  for i in range(len(valset)):

    # Get one input from the validation set
    input = torch.tensor(valset.iloc[i][1:], dtype=torch.float).to(device)
    
    # Calculate the output
    output = model(input)

    # Calculate the MSE loss
    validation_loss_point = criterion(output, torch.tensor(valset.iloc[i][0:1], dtype=torch.float).to(device))

    # update validation_loss
    validation_loss_sum += validation_loss_point.item()

  print(f'{datapoint} Validation Loss: {validation_loss_sum / len(valset):.3f}')

  torch.cuda.empty_cache()


In [26]:
def train(model, criterion, optimizer, epochs, training_set, validation_set):
  
  # Empty the cache of CUDA  
  torch.cuda.empty_cache()
  
  print('================== START TRAINING ==================')

  # Change to train mode
  model.train()

  for epoch in range(epochs):
    running_loss = 0
    for data_point in range(len(training_set)):

      # Get one input from the training set
      input = torch.tensor(training_set.iloc[data_point][1:], dtype=torch.float).to(device)

      # Calculate its corresponding output
      result = model(input)
      # print(result)

      # Calculate the MSE loss
      loss = criterion(result, torch.tensor(training_set.iloc[data_point][0:1], dtype=torch.float).to(device))
      if ((data_point+1) % 100) == 0:
        print(f'training {data_point+1} loss: {loss}')

      # Zero the gradient
      optimizer.zero_grad()
                
      # Back prop and update
      loss.backward()
      optimizer.step()

      running_loss += loss.item()

      # Validation
      if ((data_point+1) % 5000) == 0:
        # Change to evaluation mode
        model.eval()

        val(model, criterion, validation_set, data_point+1)

        # Change back to training mode
        model.train()

    print(f'[{epoch + 1}] average loss per epoch: {running_loss / len(training_set):.3f}')

    save_path = f'/content/drive/My Drive/Task4/test1/model_epoch{epoch+1}.pt'
    torch.save({'epoch': epoch+1, 'model_state_dict': model.state_dict()}, save_path)
    print(f'Saved model checkpoint to {save_path}')

In [27]:
train(model, criterion, optimizer, 5, training, validation)

================== START TRAINING ==================
training 100 loss: 0.006629479117691517
training 200 loss: 0.1727408468723297
training 300 loss: 0.17348702251911163
training 400 loss: 0.05562751740217209
training 500 loss: 0.3524382710456848
training 600 loss: 0.05999352037906647
training 700 loss: 0.0006808076286688447
training 800 loss: 0.0036711804568767548
training 900 loss: 0.010571641847491264
training 1000 loss: 0.06037874147295952
training 1100 loss: 0.07002365589141846
training 1200 loss: 0.0031184267718344927
training 1300 loss: 0.004192217253148556
training 1400 loss: 0.00508978171274066
training 1500 loss: 0.015186920762062073
training 1600 loss: 0.006078209728002548
training 1700 loss: 0.010147289372980595
training 1800 loss: 0.06858671456575394
training 1900 loss: 0.0006138374446891248
training 2000 loss: 0.001844224170781672
training 2100 loss: 0.0010619936510920525
training 2200 loss: 0.015589496120810509
training 2300 loss: 0.00087946024723351
training 2400 loss: 

In [28]:
feature_gap = pd.read_csv('train_features.csv')  # features
label_gap = pd.read_csv('train_labels.csv')  # labels


print(feature_gap.shape)
print(label_gap.shape)

# Merge two dataframes
train_gap_data = pd.merge(label_gap, feature_gap, on='Id')
print(train_gap_data.shape)
print(train_gap_data.head(10))

train_gap_data_noidsmiles = train_gap_data.drop(['Id', 'smiles'], axis=1)
print(train_gap_data_noidsmiles.head(10))

(100, 1002)
(100, 2)
(100, 1003)
      Id  homo_lumo_gap                                             smiles  \
0  50000       2.052872    C1C=c2c3ccoc3c3c4ccccc4c(cc3c2=C1)-c1scc2ccsc12   
1  50001       1.325530  c1cc([se]c1-c1sc(-c2cccc3nsnc23)c2nccnc12)-c1c...   
2  50002       1.837294  [SiH2]1C=CC=C1c1cc2cnc3c(sc4ccc5c[nH]cc5c34)c2...   
3  50003       1.388601  C1C=c2ccc3c4cocc4c4c([se]c5cc(-c6cccs6)c6nsnc6...   
4  50004       0.991851  C1c(ccc1-c1sc(-c2nccc3nsnc23)c2ccoc12)-c1scc2c...   
5  50005       1.181848  c1c[nH]c(c1)-c1sc(-c2ccc(-c3scc4[se]ccc34)c3ns...   
6  50006       1.469864  C1C(=Cc2c1c1cnc3ccc4=C[SiH2]C=c4c3c1c1c[nH]cc2...   
7  50007       1.780535  c1c[nH]c(c1)-c1ccc([nH]1)-c1sc(-c2scc3ccoc23)c...   
8  50008       2.959695        c1cc2oc3c(ccc4cc(cnc34)-c3cccc4ccccc34)c2s1   
9  50009       2.224978  c1cc2csc(-c3cc4cc5c6c[nH]cc6c6cc[se]c6c5cc4[nH...   

   feature_0000  feature_0001  feature_0002  feature_0003  feature_0004  \
0           0.0           0.0    

In [39]:
for name, param in model.named_parameters():
  print(name)
  if ('encoder_hidden_layer_2' in name) or ('encoder_hidden_layer_3' in name) or ('encoder_hidden_layer_4' in name) or ('prediction_layer' in name):
    print(name)
    param.requires_grad = True
  else:
    param.requires_grad = False

encoder_hidden_layer_1.weight
encoder_hidden_layer_1.bias
encoder_hidden_layer_2.weight
encoder_hidden_layer_2.weight
encoder_hidden_layer_2.bias
encoder_hidden_layer_2.bias
encoder_hidden_layer_3.weight
encoder_hidden_layer_3.weight
encoder_hidden_layer_3.bias
encoder_hidden_layer_3.bias
encoder_hidden_layer_4.weight
encoder_hidden_layer_4.weight
encoder_hidden_layer_4.bias
encoder_hidden_layer_4.bias
prediction_layer.weight
prediction_layer.weight
prediction_layer.bias
prediction_layer.bias


In [40]:
criterion_gap = nn.MSELoss()

optimizer_gap = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                            lr=0.00005,
                            momentum=0.9,
                            weight_decay=2e-3,#The value used in the paper is 1e-3
                            nesterov=True)

def train_gap(model, criterion, optimizer, training_set_gap):
  # Empty the cache of CUDA  
  torch.cuda.empty_cache()
  
  print("model.layer1.weight", model.encoder_hidden_layer_1.weight)
  print("model.layer2.weight", model.encoder_hidden_layer_2.weight)
  print("model.layer3.weight", model.encoder_hidden_layer_3.weight)
  print("model.layer4.weight", model.encoder_hidden_layer_4.weight)
  print("model.output.weight", model.prediction_layer.weight) 
  print('================== START TRANSFER LEARNING ==================')

  # Change to train mode
  model.train()

  running_loss = 0
  for i in range(len(training_set_gap)):
    # Get one input from the training set
    input = torch.tensor(training_set_gap.iloc[i][1:], dtype=torch.float).to(device)

    # Calculate its corresponding output
    result = model(input)
    # print(result)

    # Calculate the MSE loss
    loss = criterion(result, torch.tensor(training_set_gap.iloc[i][0:1], dtype=torch.float).to(device))

    # print(f'training {i+1} loss: {loss}')

    # Zero the gradient
    optimizer.zero_grad()
              
    # Back prop and update
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f'average loss per epoch: {running_loss / len(training_set_gap):.3f}')

  print("model.layer1.weight", model.encoder_hidden_layer_1.weight)
  print("model.layer2.weight", model.encoder_hidden_layer_2.weight)
  print("model.layer3.weight", model.encoder_hidden_layer_3.weight)
  print("model.layer4.weight", model.encoder_hidden_layer_4.weight)
  print("model.output.weight", model.prediction_layer.weight)

  return model

In [ ]:
train_gap(model, criterion_gap, optimizer_gap, train_gap_data_noidsmiles)

================== START TRANSFER LEARNING ==================
average loss per epoch: 0.061


FeatureExtract(
  (encoder_hidden_layer_1): Linear(in_features=1000, out_features=512, bias=True)
  (encoder_hidden_layer_2): Linear(in_features=512, out_features=256, bias=True)
  (encoder_hidden_layer_3): Linear(in_features=256, out_features=128, bias=True)
  (encoder_hidden_layer_4): Linear(in_features=128, out_features=64, bias=True)
  (prediction_layer): Linear(in_features=64, out_features=1, bias=True)
)

In [47]:
for i in range(900):
  train_gap(model, criterion_gap, optimizer_gap, train_gap_data_noidsmiles)

Streaming output truncated to the last 5000 lines.
        [ 0.0009,  0.0036,  0.0054,  ...,  0.0042,  0.0039, -0.0030]],
       device='cuda:0', requires_grad=True)
model.layer3.weight Parameter containing:
tensor([[ 0.0002, -0.0012,  0.0001,  ...,  0.0008,  0.0062,  0.0052],
        [-0.0017, -0.0045, -0.0015,  ..., -0.0026, -0.0019,  0.0030],
        [-0.0024,  0.0015, -0.0002,  ...,  0.0048, -0.0040,  0.0013],
        ...,
        [-0.0028,  0.0012, -0.0042,  ...,  0.0032, -0.0013, -0.0025],
        [-0.0024, -0.0009,  0.0090,  ..., -0.0057, -0.0236, -0.0048],
        [ 0.0014,  0.0032, -0.0048,  ...,  0.0049,  0.0002, -0.0042]],
       device='cuda:0', requires_grad=True)
model.layer4.weight Parameter containing:
tensor([[-5.7237e-03, -3.9983e-03,  2.8990e-04,  ..., -4.6358e-03,
         -4.8249e-03, -5.5626e-03],
        [-3.2827e-03, -5.1221e-03,  1.6641e-03,  ...,  6.1138e-03,
         -7.0742e-04, -1.3342e-03],
        [-2.5899e-03,  7.1646e-05, -6.2731e-03,  ...,  6.2866e-03,

In [53]:
model(torch.tensor(train_gap_data_noidsmiles.iloc[97][1:], dtype=torch.float).to(device))

tensor([2.0767], device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
model.eval()
with torch.no_grad():
  result = model(torch.tensor(train_gap_data_noidsmiles.iloc[98][1:], dtype=torch.float).to(device))
  print(result)
criterion(result, torch.tensor(train_gap_data_noidsmiles.iloc[98][0:1], dtype=torch.float).to(device))

tensor([1.0400], device='cuda:0')


tensor(0.3399, device='cuda:0')

In [54]:
test_feature = pd.read_csv('test_features.csv')
test_id = test_feature['Id']
print(test_id.head(10))
test_feature_noidsmiles = test_feature.drop(['Id', 'smiles'], axis=1)
print(test_feature_noidsmiles.head(10))

0    50100
1    50101
2    50102
3    50103
4    50104
5    50105
6    50106
7    50107
8    50108
9    50109
Name: Id, dtype: int64
   feature_0000  feature_0001  feature_0002  feature_0003  feature_0004  \
0           0.0           0.0           0.0           1.0           1.0   
1           0.0           0.0           0.0           0.0           0.0   
2           0.0           0.0           0.0           0.0           0.0   
3           0.0           0.0           0.0           0.0           1.0   
4           0.0           0.0           0.0           1.0           0.0   
5           0.0           0.0           0.0           1.0           0.0   
6           0.0           0.0           0.0           0.0           1.0   
7           0.0           0.0           0.0           1.0           0.0   
8           0.0           0.0           0.0           0.0           0.0   
9           0.0           0.0           0.0           0.0           0.0   

   feature_0005  feature_0006  feature_00

In [ ]:
y = []
y.append(10)
y.append(11)
y.append(12)
df = pd.DataFrame(y, columns=['y'])
print(df)
id = []
id.append(0)
id.append(1)
id.append(2)
id_df = pd.DataFrame(id, columns=['id'])
print(id_df)
det = pd.concat([id_df, df], join = 'outer', axis = 1)
print(det)
det.to_csv('/content/drive/MyDrive/test.csv', index=False, header=True)

    y
0  10
1  11
2  12
   id
0   0
1   1
2   2
   id   y
0   0  10
1   1  11
2   2  12


In [55]:
def predict(model, test_features, test_id, epoch_run):
  
  model.eval()

  y = []
  for i in range(len(test_feature)):
    input = torch.tensor(test_features.iloc[i], dtype=torch.float).to(device)

    # Calculate its corresponding output
    with torch.no_grad():
      result = model(input)
      result = result.item()
      y.append(result)

  output_df = pd.DataFrame(y, columns=['y'])

  prediction = pd.concat([test_id, output_df], join='outer', axis=1)

  prediction.to_csv('/content/drive/MyDrive/Task4/test1/submission_epoch{0}.csv'.format(epoch_run), index=False, header=True)




In [56]:
predict(model, test_feature_noidsmiles, test_id, 5)

In [ ]:
def train(model, criterion, optimizer, epochs, trainloader, valloader, testloader):

  # Create an iterator object for valloader, for selecting a random batch from val set for validation
  valloader_iterator = iter(valloader)

  # Empty the cache of CUDA  
  torch.cuda.empty_cache()
  
  print('================== START TRAINING ==================')
  # Change to train mode
  model.train()
  for epoch in range(epochs):
      running_loss = 0
      for batch_idx, (data0, data1, data2) in enumerate(tqdm(trainloader)):
          anchor, positive, negative = data0, data1, data2
          anchor = Variable(anchor)
          positive = Variable(positive)
          negative = Variable(negative)
          # print('anchor', anchor.size())
          # print('positive', positive.size())
          # print('negative', negative.size())
          
          # Calculate the output of three networks
          embedded_a, embedded_p, embedded_n = model(anchor, positive, negative)
          
          # Calculate the loss
          loss = criterion(embedded_a, embedded_p, embedded_n)
          print("mini Batch {0} Loss: {1}".format(batch_idx+1, loss.data))
          
          # Zero the gradient
          optimizer.zero_grad()
          
          # Back prop and update
          loss.backward()
          optimizer.step()
          
          # print statistics
          running_loss += loss.item()

          if batch_idx % 300 == 0 and batch_idx != 0:
            print("Training Batch: {0} | Training Loss: {1}".format(batch_idx+1, loss.data))

            ''' For Validation'''
            # Change to evaluation mode
            model.eval()

            mean_accuracy = val_accuracy(model, valloader, valloader_iterator)
            print(mean_accuracy)

            # ''' For Prediction '''
            # print('================== START PREDICTION ==================')

            # redicted_labels = np.zeros(len(testloader))
            # pred_test=[]

            # #Predict labels 1 or 0 for each test triplet
            # for batch_idx_predict_in_epoch, (data1, data2, data3) in enumerate(tqdm(testloader)):

            #     data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()

            #     # wrap in torch.autograd.Variable
            #     data1, data2, data3 = Variable(data1), Variable(data2), Variable(data3)

            #     with torch.no_grad():
            #         # compute output and loss
            #         embedded_x, embedded_y, embedded_z = model(data1, data2, data3)

            #     dist_a = F.pairwise_distance(embedded_x, embedded_y, 2)
            #     dist_b = F.pairwise_distance(embedded_x, embedded_z, 2)
            #     #print(np.squeeze(embedded_a.cpu().detach().numpy()).shape)
                

            #     pred_test.append(1*(dist_a <= dist_b))

            #     print('batch id predict in epoch: ', batch_idx_predict_in_epoch)

            # pred_test_np = []
            # for i in range(len(pred_test)):
            #   pred_test_cpu = pred_test[i].cpu().detach().numpy()
            #   pred_test_np += list(pred_test_cpu)
            # print(len(pred_test_np))
            # predicted_labels = np.hstack(pred_test_np)
            # print(predicted_labels)

            # # Write submisison file, should be saved to the current training batch ID instead of the prediction ID 
            # df = pd.DataFrame(predicted_labels)
            # df.to_csv('/content/drive/MyDrive/test8/submission_epoch{0}_batch{1}.txt'.format(epoch+1, batch_idx+1), index=False, header=None) #write CSV

            # save_path = f'/content/drive/My Drive/test8/model_epoch_{epoch+1}_batch_{batch_idx+1}.pt'
            # torch.save({'Batch id in epoch': batch_idx+1, 'model_state_dict': model.state_dict()}, save_path)
            # print("Training Batch: {0} | Model saved to: {1}".format(batch_idx+1, save_path))

            # Change back to train mode
            model.train()

            # Empty the cache of CUDA  
            torch.cuda.empty_cache()

          
      print(f'[{epoch + 1}] average loss per epoch: {running_loss / len(train_loader):.3f}')
      # # save checkpoint of model
      # if epoch % 5 == 0 and epoch > 0:

      save_path = f'/content/drive/My Drive/test9/model_epoch{epoch+1}.pt'
      torch.save({'epoch': epoch, 'model_state_dict': model.state_dict()}, save_path)
      print(f'Saved model checkpoint to {save_path}')

      ''' For Validation'''
      # Change to evaluation mode
      model.eval()

      mean_accuracy = val_accuracy(model, valloader, valloader_iterator)
      print(mean_accuracy)

      ''' For Prediction '''
      print('================== START PREDICTION ==================')

      redicted_labels = np.zeros(len(testloader))
      pred_test=[]

      #Predict labels 1 or 0 for each test triplet
      for batch_idx_predict_after_epoch, (data1, data2, data3) in enumerate(tqdm(testloader)):

          data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()

          # wrap in torch.autograd.Variable
          data1, data2, data3 = Variable(data1), Variable(data2), Variable(data3)

          with torch.no_grad():
              # compute output and loss
              embedded_x, embedded_y, embedded_z = model(data1, data2, data3)

          dist_a = F.pairwise_distance(embedded_x, embedded_y, 2)
          dist_b = F.pairwise_distance(embedded_x, embedded_z, 2)
          #print(np.squeeze(embedded_a.cpu().detach().numpy()).shape)
          

          pred_test.append(1*(dist_a <= dist_b))

          print('batch id predict after epoch: ', batch_idx_predict_after_epoch+1)

      pred_test_np = []
      for i in range(len(pred_test)):
        pred_test_cpu = pred_test[i].cpu().detach().numpy()
        pred_test_np += list(pred_test_cpu)
      len(pred_test_np)
      predicted_labels = np.hstack(pred_test_np)
      print(predicted_labels)

      #Write submisison file
      df = pd.DataFrame(predicted_labels)
      df.to_csv('/content/drive/MyDrive/test9/submission_epoch{0}.txt'.format(epoch+1), index=False, header=None) #write CSV

      # Change back to train mode
      model.train()

      # Empty the cache of CUDA  
      torch.cuda.empty_cache()

  
  print('Finished Training')
  return model

In [ ]:
# !pip install hiddenlayer

In [ ]:
# ''' Used to visualize the network structure '''
# import hiddenlayer as hl

# transforms = [hl.transforms.Prune('Constant')] # Removes Constant nodes from graph.

# graph = hl.build_graph(model, torch.zeros([10000]).to(device), transforms)
# graph.theme = hl.graph.THEMES['blue'].copy()
# graph.save('layers', format='png')

In [ ]:
class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=128
        )
        self.encoder_output_layer = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_hidden_layer = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_output_layer = nn.Linear(
            in_features=128, out_features=kwargs["input_shape"]
        )

    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed

In [ ]:
model_autoencoder = AE(input_shape=784).to(device)

In [ ]:
random_data = torch.rand((784)).to(device)
result = model(random_data)
print(result.size())

torch.Size([784])


In [ ]:
graph = hl.build_graph(model_autoencoder, torch.rand([784]).to(device), transforms)
graph.theme = hl.graph.THEMES['blue'].copy()
graph.save('layers', format='png')

In [ ]:
# Function of spliting the dataset into training set and validation set
def split_huge_file(file, out1, out2, percentage):
    """Splits a file in 2 given the approximate `percentage` to go in the large file."""
    with open(file, 'r',encoding="utf-8") as fin, \
         open(out1, 'w') as foutBig, \
         open(out2, 'w') as foutSmall:

        for line in fin:
            r = random.random() 
            if r < percentage:
                foutBig.write(line)
            else:
                foutSmall.write(line)

In [ ]:
path = '/content/drive/My Drive/'
split_huge_file(os.path.join(path, f'train_triplets.txt'), 'train_triplets_splits.txt', 'val_triplets_splits.txt', percentage=0.9)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Wed May 11 04:42:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Image loader helper function
def default_image_loader(path):
    return Image.open(path).convert('RGB')

In [ ]:
im = Image.open(r"/content/drive/My Drive/food/00003.jpg")

In [ ]:
# display(im)

In [ ]:
data = np.asarray(im)
data.shape

(329, 468, 3)

In [ ]:
# im.resize((354,242))

In [ ]:
class TripletImageLoader(torch.utils.data.Dataset):
    def __init__(self, base_path, triplets_file_name, transform=None, loader=default_image_loader):
        """ base_path: The path contains the text file of the training triplets
            triplets_file_name: The text file with each line containing three integers, 
            where integer i refers to the i-th image in the filenames file.  
            Each line contains three integers (a triplet).
            For example, the triplet "00723 00478 02630" denotes that the dish in image "00723.jpg" is more similar in taste 
            to the dish in image "00478.jpg" than to the dish in image "02630.jpg" according to a human annotator.
         """
        self.base_path = base_path  
        triplets = []
        for line in open(triplets_file_name):
            triplets.append((line.split()[0], line.split()[1], line.split()[2])) # anchor, positive, negative
        self.triplets = triplets
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        path1, path2, path3 = self.triplets[index]
        img1 = self.loader(os.path.join(self.base_path, f'{path1}.jpg'))
        img2 = self.loader(os.path.join(self.base_path, f'{path2}.jpg'))
        img3 = self.loader(os.path.join(self.base_path, f'{path3}.jpg'))
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)

        return img1, img2, img3

    def __len__(self):
        return len(self.triplets)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
device

'cuda'

In [ ]:
# Copy the test_triplets dataset to the working directory
!cp /content/drive/MyDrive/test_triplets.txt /content

In [ ]:
# Dataset and Trasformations

############# Datasets and Dataloaders ################
transform_train = transforms.Compose([
    transforms.ToTensor(), # The output of torchvision datasets are PILImage images of range [0, 1].
    transforms.Resize(IMAGE_SIZE),

    # we want our network to be robust over geometrical transformations that leave the image semantically invariant
    # transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(45),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2),

    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), #  We transform them to Tensors of normalized range [-1, 1].
    # (mean, mean, mean) , (std, std, std): output[channel] = (input[channel] - mean[channel]) / std[channel]
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(IMAGE_SIZE),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(IMAGE_SIZE),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


path = '/content/drive/MyDrive/food'
train_dataset = TripletImageLoader(path.rstrip('\n'), 'train_triplets_splits.txt', transform=transform_train)
val_dataset = TripletImageLoader(path.rstrip('\n'), 'val_triplets_splits.txt', transform=transform_val)
test_dataset = TripletImageLoader(path.rstrip('\n'), 'test_triplets.txt', transform=transform_test)

In [ ]:
len(train_dataset), len(val_dataset), len(test_dataset)

(58933, 582, 59544)

In [ ]:
test_dataset[0]

(tensor([[[ 0.6848,  0.6188,  0.6295,  ...,  0.6845,  0.6174,  0.5918],
          [ 0.7634,  0.6081,  0.6502,  ...,  0.7868,  0.7373,  0.6763],
          [ 0.7259,  0.7257,  0.6671,  ...,  0.7282,  0.7121,  0.6763],
          ...,
          [ 0.1368,  0.1779,  0.1858,  ...,  0.5491,  0.5174,  0.5192],
          [ 0.1719,  0.1433,  0.1679,  ...,  0.5418,  0.5576,  0.5111],
          [ 0.2298,  0.0134,  0.0636,  ...,  0.6480,  0.7295,  0.7176]],
 
         [[ 0.3083,  0.2423,  0.2557,  ...,  0.2424,  0.3027,  0.3446],
          [ 0.3778,  0.2235,  0.2622,  ...,  0.3537,  0.3888,  0.3711],
          [ 0.3208,  0.3215,  0.2648,  ...,  0.3097,  0.3167,  0.2928],
          ...,
          [-0.2550, -0.2139, -0.2107,  ...,  0.0953,  0.1322,  0.1737],
          [-0.2046, -0.2332, -0.2138,  ...,  0.0879,  0.1528,  0.1360],
          [-0.1141, -0.3449, -0.3147,  ...,  0.1256,  0.0728, -0.0351]],
 
         [[ 0.1512,  0.0762,  0.0626,  ..., -0.0076, -0.0635, -0.0872],
          [ 0.2222,  0.0503,

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
len(train_loader), len(val_loader), len(test_loader)

(921, 10, 931)

In [ ]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [ ]:
# Visualization of Dataset
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    plt.figure()
    plt.imshow(img.permute(1, 2, 0))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images_anchor, images_positive, images_negative = dataiter.next()

# show images
# imshow(torchvision.utils.make_grid(images_anchor))
# imshow(torchvision.utils.make_grid(images_positive))
# imshow(torchvision.utils.make_grid(images_negative))

In [ ]:
images_anchor

tensor([[[[ 0.6884,  0.6038,  0.7176,  ...,  0.8562,  0.6681,  0.6141],
          [ 0.6604,  0.6287,  0.6395,  ...,  0.7792,  0.7544,  0.7447],
          [ 0.6743,  0.6843,  0.6265,  ...,  0.7189,  0.7146,  0.7013],
          ...,
          [ 0.4680,  0.4406,  0.4628,  ...,  0.7125,  0.7397,  0.8157],
          [ 0.3393,  0.3476,  0.4362,  ...,  0.7232,  0.7844,  0.8184],
          [ 0.3865,  0.4128,  0.5302,  ...,  0.6692,  0.7686,  0.7089]],

         [[ 0.1080,  0.0235,  0.1323,  ...,  0.5122,  0.3442,  0.3048],
          [ 0.0800,  0.0474,  0.0532,  ...,  0.4193,  0.4160,  0.4125],
          [ 0.0898,  0.0983,  0.0382,  ...,  0.3534,  0.3613,  0.3512],
          ...,
          [ 0.4019,  0.3458,  0.3217,  ...,  0.6948,  0.7417,  0.8163],
          [ 0.3007,  0.2792,  0.3206,  ...,  0.6954,  0.7830,  0.8350],
          [ 0.3882,  0.3781,  0.4382,  ...,  0.6367,  0.7788,  0.7444]],

         [[-0.2449, -0.3295, -0.2058,  ...,  0.4415,  0.2583,  0.1909],
          [-0.2730, -0.3027, -

In [ ]:
'''
Data loaded
'''
print('================== DATA LOADED ==================')

================== DATA LOADED ==================


In [ ]:
'''
Construct a triplet net, where three CNNs with shared weights and a modified 1024 fully connected output are working in parallel
The 1024 dimension output were used to calculate the Euclidean distance to construnct the Triplet Margin loss
'''
#########################NET##############################

#The CNNs with shared weights and the fully connected layer replaced
def FeatureExtractNET(**kwargs):
    """
    Construct a pretrained ResNet-18 model
    Returns: The CNN for feature extraction with a fully connected layer
    """
    model = models.resnet18(pretrained=True)

    return EmbeddingNet(model)

#The CNN used by Triplet Net with 'model' as its backbone and a final fully connected Layer
class EmbeddingNet(nn.Module):
    """ 
    EmbeddingNet using the specified model in FeatureExtractNET(). It replace the last fully connected layer of the specified model with
    another fully connected layer of 1024 demensions' output. 
    """

    def __init__(self, resnet):
        """Initialize EmbeddingNet model."""
        super(EmbeddingNet, self).__init__()

        # Everything excluding the last linear layer
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        num_ftrs =  resnet.fc.in_features

        # Map from the CNN's output dimension(512) to the feature space of dimension 1024
        self.fc1 = nn.Sequential(nn.Linear(num_ftrs, 1024))
        # nn.BatchNorm1d(1024),
        # nn.Dropout(0.3)

    def forward(self, x):
        """Forward pass of EmbeddingNet."""
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out
    
#The overall network consisting of three embedding nets with shared weights and three 1024 dimension outputs
class TripletNet(nn.Module):
    """Triplet Network."""

    def __init__(self, embeddingnet):
        """Triplet Network Builder."""
        super(TripletNet, self).__init__()
        self.embeddingnet = embeddingnet

    def forward(self, a, p, n):
        """Forward pass."""
        # anchor
        embedded_a = self.embeddingnet(a)

        # positive examples
        embedded_p = self.embeddingnet(p)

        # negative examples
        embedded_n = self.embeddingnet(n)

        return embedded_a, embedded_p, embedded_n

In [ ]:
net = TripletNet(FeatureExtractNET())

#Move the net to GPU for training
print("==> Initialize CUDA support for TripletNet model ...")
net = torch.nn.DataParallel(net).cuda()
# cudnn.benchmark = True
net

==> Initialize CUDA support for TripletNet model ...


DataParallel(
  (module): TripletNet(
    (embeddingnet): EmbeddingNet(
      (features): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (4): Sequential(
          (0): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inplace=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (1): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# batch = next(iter(train_loader))
# batch[0], batch[1], batch[2] = batch[0].cuda(), batch[1].cuda(), batch[2].cuda()
# net(batch[0],batch[1],batch[2])[0].size()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# from torch.autograd import Variable
# for epoch in range(1):

#         running_loss = 0.0
#         loss_train = 0.0
#         for batch_idx, (data1, data2, data3) in enumerate(train_loader):

# #             if is_gpu:
# #                 data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()

#             # wrap in torch.autograd.Variable
#             data1, data2, data3 = Variable(
#                 data1), Variable(data2), Variable(data3)
#             print('anchor', data1.size())
#             print('positive', data2.size())
#             print('negative', data3.size())

#             # compute output and loss
#             embedded_a, embedded_p, embedded_n = net(data1, data2, data3)
#             loss = criterion(embedded_a, embedded_p, embedded_n)
#             print(loss)

#             # compute gradient and do optimizer step
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             # print the loss
#             running_loss += loss.data

# #             loss_train_cls = torch.sum(
# #                 1 * (criterion_val(embedded_a, embedded_p,
# #                                    embedded_n) > 0)) / train_batch_size  # CHANGED, MAY NEED TO REVERT BACK

# #             loss_train += loss_train_cls.data

#             if batch_idx % 30 == 0:
#                 print("mini Batch Loss: {}".format(loss.data))

In [ ]:
'''
Network constructed
'''
print('================== NETWORK CONSTRUCTED ==================')

================== NETWORK CONSTRUCTED ==================


In [ ]:
'''Used to test the accuracy function'''
# import torch
# import torch.nn as nn
# pdist = nn.PairwiseDistance(p=2)
# input1 = torch.randn(64, 1024)
# input2 = torch.randn(64, 1024)
# input3 = torch.randn(64, 1024)
# print(input1.size())
# print(input2.size())
# print(input3.size())
# dist1 = pdist(input1, input2)
# dist2 = pdist(input1, input3)
# print(dist1.size())
# print(dist2.size())
# pred = dist1 - dist2
# print(pred.size())
# sum = 0
# for i in range(pred.size()[0]):
#   if pred[i] < 0:
#     sum+=1
# print(sum/pred.size()[0])
# print((pred < 0).sum()*1.0/pred.size()[0])

'Used to test the accuracy function'

In [ ]:
# import random
# for i in range(10):
#   a = random.randint(0,10)
#   print(a)

In [ ]:
import torch.nn.functional as F
from torch.autograd import Variable

'''
Given an input of a tensor with multiple sets of triplets in the validation set, where dista is expected to smaller than distb, calculate
how many dista are exactly smaller than distb
'''
def accuracy(dista, distb):
    pred = (dista - distb).cpu().data
    return (pred < 0).sum()*1.0/dista.size()[0]

'''
Return the accuracy of a random batch from the validation set
'''
def val_accuracy(trainednet, valloader, valloader_iter):

  try:
    data1, data2, data3 = next(valloader_iter)
  except StopIteration:
    valloader_iterator = iter(valloader)
    data1, data2, data3 = next(valloader_iterator)

  data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()

  # wrap in torch.autograd.Variable
  data1, data2, data3 = Variable(data1), Variable(data2), Variable(data3)

  with torch.no_grad():
    # compute output and loss
    embedded_x, embedded_y, embedded_z = trainednet(data1, data2, data3)
    dist_a = F.pairwise_distance(embedded_x, embedded_y, 2)
    dist_b = F.pairwise_distance(embedded_x, embedded_z, 2)
    print('dist a: {0}, dist b: {1}'.format(dist_a, dist_b))
    batch_accuracy = accuracy(dist_a, dist_b)
    print('random batch accuracy: {0} '.format(batch_accuracy))

  # mean_accuracy = sum_accuracy / num_batch_evaluated
  return batch_accuracy

In [ ]:
# torch.cuda.empty_cache()
# batch = next(iter(train_loader))
# batch[0], batch[1], batch[2] = batch[0].cuda(), batch[1].cuda(), batch[2].cuda()
# embedded_x, embedded_y, embedded_z = net(batch[0],batch[1],batch[2])
# dist_a = F.pairwise_distance(embedded_x, embedded_y, 2)
# dist_b = F.pairwise_distance(embedded_x, embedded_z, 2)
# accuracy = accuracy(dist_a, dist_b)
# print(accuracy)

In [ ]:
trained_net = train(net, criterion, optimizer, 1, train_loader, val_loader, test_loader)

================== START TRAINING ==================


  0%|          | 0/921 [00:00<?, ?it/s]

mini Batch 1 Loss: 4.577956676483154


  0%|          | 1/921 [00:42<10:58:57, 42.98s/it]

mini Batch 2 Loss: 4.295666694641113


...

100%|█████████▉| 930/931 [43:39<00:01,  1.29s/it]

batch id predict after epoch:  930


100%|██████████| 931/931 [43:39<00:00,  2.81s/it]

batch id predict after epoch:  931


[1 0 0 ... 0 0 0]
Finished Training


In [ ]:
trained_net.eval()

In [ ]:
import torch
# load model from checkpoint
model_loaded = TripletNet(FeatureExtractNET())
checkpoint = torch.load('/content/drive/MyDrive/test3/model_epoch_5.pt')
model_loaded.load_state_dict(checkpoint, strict=False)

model_loaded.to(device)
model_loaded.eval()

In [ ]:
with torch.no_grad():
  valloader_iterator = iter(train_loader)
  mean_accuracy = val_accuracy(model_loaded, train_loader, valloader_iterator)
  print(mean_accuracy)

In [ ]:
count = len(open("/content/drive/MyDrive/test3/submission_1epoch.txt",'rU').readlines())
print(count)